In [74]:
import os
import pandas as pd
import time
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense

from keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_squared_error

In [75]:
import os
import pandas as pd

folder_path = r"C:\Users\Admin\Desktop\forthSem\train"

X_data = []
y_data = []

for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        
        data = pd.read_csv(file_path)
        
        X_file = data[['vdd','pd', 'vinp','temperature','process']] 
        y_file = data['vinn']
        
        X_data.append(X_file)
        y_data.append(y_file)

X = pd.concat(X_data, ignore_index=True)
y = pd.concat(y_data, ignore_index=True)


print("Shape of X:", X.shape)
print("Shape of y:", y.shape)




Shape of X: (2100000, 5)
Shape of y: (2100000,)


In [76]:
# X= data[['vdd','pd', 'vinp','temperature','process']]
X

,vdd,pd,vinp,temperature,process
0,0.0000,0.0000,0.000000,-15,fastnfastp
1,0.0033,0.0033,0.000000,-15,fastnfastp
2,0.0066,0.0066,0.000000,-15,fastnfastp
3,0.0099,0.0099,0.000000,-15,fastnfastp
4,0.0132,0.0132,0.000000,-15,fastnfastp
...,...,...,...,...,...
2099995,3.0000,3.0000,1.638227,85,typical
2099996,3.0000,3.0000,1.640581,85,typical
2099997,3.0000,3.0000,1.642936,85,typical
2099998,3.0000,3.0000,1.645291,85,typical


In [77]:
# y = data['vinn']
y

0          0.000000
1          0.001346
2          0.002692
3          0.004038
4          0.005384
             ...   
2099995    0.883872
2099996    0.883918
2099997    0.883964
2099998    0.884009
2099999    0.884055
Name: vinn, Length: 2100000, dtype: float64

In [78]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K


# Perform label encoding for the 'process' column
label_encoder = LabelEncoder()
X['process'] = label_encoder.fit_transform(X['process'])


# Convert DataFrame to numpy array
X = X.values
y = y.values

# Convert data type to float32
X = X.astype(np.float32)
y = y.astype(np.float32)

# Preprocess the input features (if necessary)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

# Make predictions
predictions = model.predict(X_test)


Epoch 1/10


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


AttributeError: Exception encountered when calling Dense.call().

[1mmodule 'keras.src.backend' has no attribute 'floatx'[0m

Arguments received by Dense.call():
  • inputs=tf.Tensor(shape=(32, 5), dtype=float32)

In [45]:
from sklearn.preprocessing import  LabelEncoder

label_encoder =LabelEncoder()

X.loc[:, 'process'] = label_encoder.fit_transform(X['process'])

In [50]:
import numpy as np

X = X.values.astype(np.float32)
y = y.values.astype(np.float32)


In [51]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [52]:
model = Sequential()
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))


In [53]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [55]:
model.fit(X, y, epochs=10, batch_size=32, verbose=1)

Epoch 1/10


AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:


# Define checkpoint to save the best model
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

# Train the model
history = model.fit(X_scaled, y, validation_data=(X_test_scaled, y_test), epochs=50, batch_size=32, callbacks=[checkpoint])

# Load the best model from the checkpoint
best_model = load_model('best_model.h5')

# Predict using the best model
y_pred = best_model.predict(X_test_scaled)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

In [4]:
import numpy as np
import pickle
import concurrent.futures

def import_set(file_path):
    with open(file_path, 'rb') as file:
        my_set = pickle.load(file)
    return my_set

def evaluate_function(function_code, kwargs):
    exec(function_code)
    results = []
    for i in range(1, 21):
        function_name = f"decision_tree_rule_{i}"
        function = locals()[function_name]
        result = function(**kwargs) or 0
        results.append(result)
    return results

def evaluate_all_conditions_parallel(input_parameters, function_code):
    results = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_param = {executor.submit(evaluate_function, function_code, params): params for params in input_parameters}
        for future in concurrent.futures.as_completed(future_to_param):
            params = future_to_param[future]
            try:
                result = future.result()
                results.append(result)
            except Exception as exc:
                print(f"Function execution generated an exception: {exc}")
                results.append([])  # Append empty list as placeholder for failed execution
    return results

import_file_path = r'exported_set.pkl'
function_file_path = r"TreeFunction.py"

input_parameters = import_set(import_file_path)
vinp_value = 1.638227 
pd_value = 3.0
vdd_value = 3.0
process_value = 3
temperature_value = 85
input_values = {'vinp': vinp_value, 'pd': pd_value, 'vdd': vdd_value, 'process': process_value, 'temperature': temperature_value}

with open(function_file_path, 'r') as file:
    functions_code = file.read()

# Convert input_parameters to a list of dictionaries
input_parameters_list = [input_values] * len(input_parameters)

result_list = evaluate_all_conditions_parallel(input_parameters_list, functions_code)

print("Length:", len(result_list))
print("Results:", result_list)
print("Average:", np.nanmean(result_list))


Length: 5
Results: [[0.768, 0.768, 0.768, 0.765, 0.767, 0.767, 0.767, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.767], [0.768, 0.768, 0.768, 0.765, 0.767, 0.767, 0.767, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.767], [0.768, 0.768, 0.768, 0.765, 0.767, 0.767, 0.767, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.767], [0.768, 0.768, 0.768, 0.765, 0.767, 0.767, 0.767, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.767], [0.768, 0.768, 0.768, 0.765, 0.767, 0.767, 0.767, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.767]]
Average: 0.7676499999999998


In [5]:


import numpy as np
import pickle

def import_set(file_path):
    with open(file_path, 'rb') as file:
        my_set = pickle.load(file)
    return my_set

import_file_path = r'exported_set.pkl'


def evaluate_all_conditions(**kwargs):
    results = []

    function_file_path = r"TreeFunction.py"

    with open(function_file_path, 'r') as file:
        functions_code = file.read()

    for i in range(1,21):
        function_name = f"decision_tree_rule_{i}"
        exec(functions_code)
        function = locals()[function_name]
        result = function(**kwargs) or 0
        results.append(result)

    return results


input_parameters = import_set(import_file_path)

print("Imported Set:", input_parameters)

vinp_value = 1.638227 
pd_value = 3.0
vdd_value = 3.0
process_value = 3
temperature_value = 85
input_values = {'vinp': vinp_value, 'pd': pd_value, 'vdd': vdd_value, 'process':process_value,'temperature':temperature_value}

result_list = evaluate_all_conditions(**input_values)

print("Length:", len(result_list))
print("Results:", result_list)
print("Average:", np.average(result_list))


Imported Set: {'pd', 'process', 'vdd', 'vinp', 'temperature'}
Length: 20
Results: [0.768, 0.768, 0.768, 0.765, 0.767, 0.767, 0.767, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.768, 0.767]
Average: 0.76765
